In [1]:
import os
import streamlit as st
import pickle
import time
import langchain
from langchain import OpenAI
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.chains.qa_with_sources.loading import load_qa_with_sources_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import UnstructuredURLLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
import requests
from bs4 import BeautifulSoup

In [2]:
import pandas as pd
key= pd.read_csv("key.txt")

In [3]:
llm= OpenAI(temperature =0.9 , max_tokens=500)

In [7]:
!pip install python-magic-bin


[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
!pip install python-libmagic

  Using cached python_libmagic-0.4.0-py3-none-any.whl
  Using cached cffi-1.7.0.tar.gz (400 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py clean for cffi
Failed to build cffi
  Attempting uninstall: cffi
    Found existing installation: cffi 1.15.1
    Uninstalling cffi-1.15.1:
      Successfully uninstalled cffi-1.15.1
  Running setup.py install for cffi: started
  Running setup.py install for cffi: finished with status 'error'
  Rolling back uninstall of cffi
  Moving to c:\users\dell\appdata\local\programs\python\python310\lib\site-packages\_cffi_backend.cp310-win_amd64.pyd
   from C:\Users\DELL\AppData\Local\Temp\pip-uninstall-g2bfqt9e\_cffi_backend.cp310-win_amd64.pyd
  Moving to c:\users\dell\appdata\local\programs\python\python310\lib\site-packages\cffi-1.15.1.dist-info\
   from C:\Users\DELL\AppData\Local\Programs\Python\Python310\Lib\site-packages\~ffi-1.15.1.dist-info
  Moving to c:\users\dell\appda

  error: subprocess-exited-with-error
  
  python setup.py bdist_wheel did not run successfully.
  exit code: 1
  
  [26 lines of output]
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-3.10
  creating build\lib.win-amd64-3.10\cffi
  copying cffi\api.py -> build\lib.win-amd64-3.10\cffi
  copying cffi\backend_ctypes.py -> build\lib.win-amd64-3.10\cffi
  copying cffi\cffi_opcode.py -> build\lib.win-amd64-3.10\cffi
  copying cffi\commontypes.py -> build\lib.win-amd64-3.10\cffi
  copying cffi\cparser.py -> build\lib.win-amd64-3.10\cffi
  copying cffi\ffiplatform.py -> build\lib.win-amd64-3.10\cffi
  copying cffi\lock.py -> build\lib.win-amd64-3.10\cffi
  copying cffi\model.py -> build\lib.win-amd64-3.10\cffi
  copying cffi\recompiler.py -> build\lib.win-amd64-3.10\cffi
  copying cffi\setuptools_ext.py -> build\lib.win-amd64-3.10\cffi
  copying cffi\vengine_cpy.py -> build\lib.win-amd64-3.10\cffi
  copying cffi\vengine_gen.py -> build

In [10]:
loaders= UnstructuredURLLoader(urls=["https://www.livemint.com/news/india/uttarkashi-tunnel-collapse-silkyara-ndrf-deploys-team-rescue-mission-41-workers-top-10-updates-pushkar-singh-dhami-11700703183514.html",
     "https://www.indiatvnews.com/sports/cricket/india-s-schedule-after-world-cup-2023-final-loss-india-vs-australia-t20i-series-t20-world-cup-2024-2023-11-20-903594"]
)
data=loaders.load()


2023-11-28 00:06:21.788 WARNING unstructured: libmagic is unavailable but assists in filetype detection on file-like objects. Please consider installing libmagic for better results.
2023-11-28 00:06:21.788 ERROR   langchain.document_loaders.url: Error fetching or processing https://www.livemint.com/news/india/uttarkashi-tunnel-collapse-silkyara-ndrf-deploys-team-rescue-mission-41-workers-top-10-updates-pushkar-singh-dhami-11700703183514.html, exception: Invalid file. The FileType.UNK file type is not supported in partition.
2023-11-28 00:06:21.958 WARNING unstructured: libmagic is unavailable but assists in filetype detection on file-like objects. Please consider installing libmagic for better results.
2023-11-28 00:06:21.958 ERROR   langchain.document_loaders.url: Error fetching or processing https://www.indiatvnews.com/sports/cricket/india-s-schedule-after-world-cup-2023-final-loss-india-vs-australia-t20i-series-t20-world-cup-2024-2023-11-20-903594, exception: Invalid file. The FileT

In [11]:
data

[]

In [12]:
urls=["https://www.livemint.com/news/india/uttarkashi-tunnel-collapse-silkyara-ndrf-deploys-team-rescue-mission-41-workers-top-10-updates-pushkar-singh-dhami-11700703183514.html",
     "https://www.indiatvnews.com/sports/cricket/india-s-schedule-after-world-cup-2023-final-loss-india-vs-australia-t20i-series-t20-world-cup-2024-2023-11-20-903594"]
responses=[]

for url in urls:
    response=requests.get(url)
    if response.status_code==200:
        soup = BeautifulSoup(response.text, 'html.parser')
        text = soup.get_text()
        responses.append((text, response.url))
        
    else:
        print(f"Failed to retrieve the URL. Url={url} Status_code :{response.status_code}")

In [13]:
import string 
punct=string.punctuation
seps=[]
for i in punct:
    seps.append(i)

In [14]:
text_splitter=RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", "?","." ]+seps+[" "],
    chunk_size=1000,
    chunk_overlap=200
)